In [2]:
!cat workshop/safe-smart-contracts-workshop-08-2015/example-05-merkletree.py

from ethereum import tester
from ethereum import utils
from ethereum import slogging
import rlp

file_chunks = [
 "A  purely peer-to-peer",
 "version of electronic",
 "cash would allow online",
 "payments to be sent directly",
 "from one party to another",
 "without going through a",
 "financial institution.",
 "Digital signatures provide",
 "part of the solution, but",
 "the main benefits are lost",
 "if a trusted third party",
 "is still required to prevent",
 "double-spending. We propose",
 "a solution to the double",
 "-spending problem using a",
 "peer-to-peer network."]

zfill = lambda s: (32-len(s))*'\x00' + s
file_chunks = map(zfill, file_chunks)

contract_code = """

root = 0x{}

macro hash_node($h, $sibling, $bit):
    if $bit == 0:
        sha3([$h, $sibling], items=2)
    else:
        sha3([$sibling, $h], items=2)

def check_index(x:bytes32, bits:uint8[4], siblings:bytes32[4]):
    h = hash_node(x, siblings[0], bits[0])
    h = hash_node(h, siblings[1], bits[1])
    h = ha

In [3]:
from ethereum import tester
from ethereum import utils
from ethereum import slogging
import rlp

file_chunks = [
 "A  purely peer-to-peer",
 "version of electronic",
 "cash would allow online",
 "payments to be sent directly",
 "from one party to another",
 "without going through a",
 "financial institution.",
 "Digital signatures provide",
 "part of the solution, but",
 "the main benefits are lost",
 "if a trusted third party",
 "is still required to prevent",
 "double-spending. We propose",
 "a solution to the double",
 "-spending problem using a",
 "peer-to-peer network."]

zfill = lambda s: (32-len(s))*'\x00' + s
file_chunks = map(zfill, file_chunks)

contract_code = """

root = 0x{}

macro hash_node($h, $sibling, $bit):
    if $bit == 0:
        sha3([$h, $sibling], items=2)
    else:
        sha3([$sibling, $h], items=2)

def check_index(x:bytes32, bits:uint8[4], siblings:bytes32[4]):
    h = hash_node(x, siblings[0], bits[0])
    h = hash_node(h, siblings[1], bits[1])
    h = hash_node(h, siblings[2], bits[2])
    h = hash_node(h, siblings[3], bits[3])
    if h == root: 
        return(1)
    else: 
        return(0)
"""

# Build the merkle tree
layer_1 = [utils.sha3(file_chunks[2*i+0] + file_chunks[2*i+1])
           for i in range(8)]
layer_2 = [utils.sha3(layer_1[2*i+0] + layer_1[2*i+1])
           for i in range(4)]
layer_3 = [utils.sha3(layer_2[2*i+0] + layer_2[2*i+1])
           for i in range(2)]
root_hash = utils.sha3(layer_3[0] + layer_3[1])


def index_to_bits(ind):
    bits = []
    for i in range(4):
        bits.append(ind % 2)
        ind /= 2
    return bits

def get_siblings(bits):
    assert len(bits) == 4
    if bits[3] == 0: sibling3 = layer_3[1]
    else: sibling3 = layer_3[0]

    offset = bits[3]*2
    if bits[2] == 0: sibling2 = layer_2[offset+1]
    else: sibling2 = layer_2[offset]

    offset = 2*offset + bits[2]*2
    if bits[1] == 0: sibling1 = layer_1[offset+1]
    else: sibling1 = layer_1[offset]

    offset = 2*offset + bits[1]*2
    if bits[0] == 0: sibling0 = file_chunks[offset+1]
    else: sibling0 = file_chunks[offset]

    return [sibling0, sibling1, sibling2, sibling3]

s = tester.state()
c = s.abi_contract(contract_code.format(root_hash.encode('hex')))

def test_index(ind):
    bits = index_to_bits(ind)
    chunk = file_chunks[ind]
    siblings = get_siblings(bits)
    assert c.check_index(chunk, bits, siblings) == 1

test_index(0)
test_index(1)
test_index(2)


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/ubuntu/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)
No handlers could be found for logger "eth.pow"


In [4]:
print test_index(0)
print test_index(1)
print test_index(2)


None
None
None


In [5]:
index_to_bits(7)

[1, 1, 1, 0]

In [6]:
index_to_bits(11)

[1, 1, 0, 1]